In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import linregress
from scipy import stats
import cartopy.crs as ccrs
import cartopy.feature as cfeature


plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.unicode_minus'] = False  


xls_new = pd.ExcelFile(r'GPP_all.xlsx')


dfs_new = [xls_new.parse(x) for x in xls_new.sheet_names]


def check_recovery(row):
    recovery_status = 2
    year_value = None
    
    for i in range(len(row)):
        if (row[i:i+2] > 0).all():
            recovery_status = 1
            year_value = i+1
            break

    if recovery_status == 2:

        if (row[-4:] < 0).all() and (row[-4:].diff().sum() <= 0):
            recovery_status = 3
            year_value = 999
        if recovery_status == 2:
            year_value = 999  #

    return recovery_status, year_value


label=['Pre-2010 NDVI','Post-2010 NDVI','All NDVI']


for i, df in enumerate(dfs_new, start=1):
    dNBR=df['dNBR']


    df = df.iloc[:, -8:].sub(df.iloc[:, -9], axis=0) 
    

    df[['Recovery', 'Year']] = df.apply(check_recovery, axis=1, result_type='expand').astype(int)
    
    df['dNBR']=dNBR
    
    df=df[df['Recovery']==1]
    
    print(df['dNBR'].max())
    print(len(df['dNBR']))


    df_mean = df.groupby('Year')['dNBR'].mean().reset_index()


    x = df_mean['Year']
    y = df_mean['dNBR']


    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    line = slope * x + intercept
    

    print(f"Dataset {i} - p-value: {p_value:.5f}, R²: {r_value**2:.5f}")

    x_fit = np.linspace(x.min(), x.max(), 100)
    y_fit = slope * x_fit + intercept


    mean_x = np.mean(x)
    n = len(x)
    t = stats.t.ppf(0.975, n-2)
    s_err = np.sum(np.power(y - line, 2)) / (n-2)
    conf = t * np.sqrt(s_err * (1.0/n + (np.power((x - mean_x), 2) / np.sum(np.power((x - mean_x), 2)))))


    conf_fit = t * np.sqrt(s_err * (1.0/n + (np.power((x_fit - mean_x), 2) / np.sum(np.power((x - mean_x), 2)))))

    upper = y_fit + conf_fit
    lower = y_fit - conf_fit


    plt.figure(figsize=(5, 4))
    plt.scatter(x, y,label='NDVI')
    plt.plot(x_fit, y_fit, color='red')
    plt.fill_between(x_fit, lower, upper, color='b', alpha=0.2)
    plt.xlabel('Recovery years',fontsize=15, weight='semibold')
    plt.ylabel('Mean dNBR',fontsize=15, weight='semibold')
    
    plt.ylim(0,0.26)
    
    plt.xticks(fontproperties='Arial',fontsize=15, weight='normal')
    plt.yticks(fontproperties='Arial', fontsize=15, weight='normal')


    plt.text(0.05, 0.95, label[i-1], transform=plt.gca().transAxes,
             fontsize=15, verticalalignment='top', weight='normal')
    

    plt.legend().set_visible(False)

#     plt.savefig(f"dNBRYear_GPP_{i}.jpg",bbox_inches = 'tight',dpi=600)
    plt.show()